# Canadian median income in constant (2018) dollars

Below you can explore median income in 2018 dollars for different regions in Canada between 1975-2017.  The data is based on [stats provided by Statistics Canada](https://doi.org/10.25318/1110023901-eng).

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Dropdown, Checkbox, IntSlider
import logging
import sys # for relative local imports

sys.path.append('..')

# local imports
import wrangling
from load_data import median_income_dropdown_values
from visualize import format_scatter_title, create_scatter_plot

logger = logging.getLogger(__name__)
FORMAT = "[%(filename)s:%(lineno)s - %(funcName)20s() ] %(message)s"
logging.basicConfig(format=FORMAT)
logger.setLevel(logging.DEBUG)

In [2]:
# 5.5s read time
#load the data
path = r"../data/raw/11100239.csv"
df = pd.read_csv(path, low_memory=False)

In [3]:
cols_to_keep = ['REF_DATE', 
                'GEO', 
                'Sex', 
                'Age group', 
                'Income source',
                'Statistics',
                'SCALAR_FACTOR', 
                'VALUE', 
                 ]

provinces = ['Atlantic provinces', 'Newfoundland and Labrador',
                'Prince Edward Island', 'Nova Scotia', 'New Brunswick', 'Quebec',
                'Ontario', 'Prairie provinces', 'Manitoba', 'Saskatchewan',
                'Alberta', 'British Columbia']
cities = ['Québec, Quebec',
                'Montréal, Quebec', 'Ottawa-Gatineau, Ontario/Quebec',
                'Toronto, Ontario', 'Winnipeg, Manitoba', 'Calgary, Alberta',
                'Edmonton, Alberta', 'Vancouver, British Columbia']

initial_sex = "Females"
initial_geo = cities[-1]
initial_age = median_income_dropdown_values["Age group"][2]

In [4]:
# params
year = 2017
income_source = "Total income"
income_to_plot = 'Median income (excluding zeros)'

In [5]:
plot_data = wrangling.subset_for_scatter_plot(
    df, year, income_source, income_to_plot, cols_to_keep)

In [6]:
sex_selector = widgets.SelectMultiple(
    options=median_income_dropdown_values.get("Sex"),
    value=(initial_sex,),
    #rows=10,
    description='Sex',
    disabled=False
)
geo_selector = widgets.SelectMultiple(
    options=median_income_dropdown_values.get("GEO"),
    value = (initial_geo,),
    description = "Location"
)
age_dropdown = widgets.Dropdown(
    options=median_income_dropdown_values.get("Age group"),
    value=initial_age,
    description="Age group")

In [7]:



scatter_plot = create_scatter_plot(
    plot_data,
    sex_selector.value,
    age_dropdown.value,
    geo_selector.value)
scatter_plot = go.FigureWidget(scatter_plot)

In [8]:
def clear_plot_data(plot):
    # we have to first delete the data in the plot
    plot.data = []
    return plot

def update_plot_data(old_plot, new_plot):
    for trace in new_plot.data:
        old_plot.add_trace(trace)
    return old_plot
    
def update_scatter_plot(change):
    global scatter_plot
    global plot_data
    age = age_dropdown.value
    geo = geo_selector.value
    sex = sex_selector.value

    # this is the current approach which doesn't work because it points to a new object
    # scatter_plot = create_scatter_plot(df, sex, age, geo)
    
    # To display the plot, we have to operate on the 
    # original plot object instead of creating a new object
    scatter_plot = clear_plot_data(scatter_plot)
    new_plot = create_scatter_plot(plot_data, sex, age, geo)
    scatter_plot = update_plot_data(scatter_plot, new_plot)
    scatter_plot.update_layout(title=format_title(sex, age))
    scatter_plot.update_layout(showlegend=True)
    scatter_plot.update_layout(legend_title="Location")


age_dropdown.observe(update_scatter_plot, 'value')
geo_selector.observe(update_scatter_plot, 'value')
sex_selector.observe(update_scatter_plot, 'value')

left_selections = VBox([age_dropdown, sex_selector])
all_selections = HBox([left_selections, geo_selector])
VBox([all_selections, scatter_plot])